In [13]:
import netgen.gui
from ngsolve import *
from netgen.geom2d import unit_square
import numpy as np
from math import pi

In [14]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
fes = H1(mesh, order=3, dirichlet='t|b|l|r')
u,v =fes.TnT()

In [15]:
x_0 = 0.5
y_0 = 0.5
maxind = 20
omega = []
sigma_sqared = 0.01
factor = 10

f = LinearForm(fes)
f += exp(-factor*((x-x_0)**2/(2*sigma_sqared) + (y-y_0)**2/(2*sigma_sqared)))* v * dx
f.Assemble()

In [16]:
for i in range(0,maxind):
    omega.append((i)*0.005*(20*pi)+60)

In [17]:
norm_of_solutions = [] # sqared l2 norm actually
V = []
z = GridFunction(fes)
tmp = GridFunction(fes)
for w in omega:
    a = BilinearForm(fes)
    a += (grad(u)*grad(v) - w*w *u*v) * dx
    a.Assemble()
    gfu = GridFunction(fes)
    gfu.vec.data = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky") * f.vec
    nof = Integrate(gfu*gfu, mesh)
    norm_of_solutions += [nof]
    if V==[]:
        z.vec.data = gfu.vec.data
    else:
        tmp.Set(0)
        for zeta in V:   
            a = Integrate(zeta*zeta+grad(zeta)*grad(zeta), mesh)
            b = Integrate(gfu*zeta + grad(gfu)*grad(zeta), mesh)
            ## attention: multiplication with scalar only allowed from left side
            tmp.vec.data += float(Integrate(gfu*zeta + grad(gfu)*grad(zeta), mesh)/
                    Integrate(zeta*zeta+grad(zeta)*grad(zeta), mesh))*zeta.vec.data 
        z.vec.data = gfu.vec.data-tmp.vec.data
    z.vec.data = float(1./sqrt(Integrate(z*z+grad(z)*grad(z), mesh)))*z.vec.data
    V += [z]

In [22]:
j = 0
for zeta in V:
    print(Integrate(zeta*zeta+grad(zeta)*grad(zeta), mesh))

1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
[<ngsolve.comp.GridFunction object at 0x7f41bee3f570>, <ngsolve.comp.GridFunction object at 0x7f41bee3f570>, <ngsolve.comp.GridFunction object at 0x7f41bee3f570>, <ngsolve.comp.GridFunction object at 0x7f41bee3f570>, <ngsolve.comp.GridFunction object at 0x7f41bee3f570>, <ngsolve.comp.GridFunction object at 0x7f41bee3f570>, <ngsolve.comp.GridFunction object at 0x7f41bee3f570>, <ngsolve.comp.GridFunction object at 0x7f41bee3f570>, <ngsolve.comp.GridFunction object at 0x7f41bee3f570>, <ngsolve.comp.GridFunction object at 0x7f41bee3f570>, <ngsolve.comp.GridFunction object at 0x7f41bee3f570>, <ngsolve.comp.GridFunctio

In [31]:
print(V[1,1])

TypeError: list indices must be integers or slices, not tuple